In [170]:
import warnings, sys, os, re
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
# from sklearn.preprocessing import normalize

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader

## Model

In [160]:
class CNN_GRU(nn.Module):
    def __init__(self):
        super(CNN_GRU, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, stride=1, padding=1).cuda()
        self.conv2 = nn.Conv2d(32, 16, 3, stride=1, padding=1).cuda()
        self.relu = nn.ReLU().cuda()
        self.maxpool = nn.MaxPool2d((2,2)).cuda()
        self.fc1 = nn.Linear(8,4).cuda()
        self.fc2 = nn.Linear(4,1).cuda()
        self.gru1 = nn.GRU(input_size=360,hidden_size=360,num_layers =10,dropout = 0.2).cuda()
        self.gru2 = nn.GRU(input_size=360,hidden_size=180,num_layers =10,dropout = 0.2).cuda()
        self.gru3 = nn.GRU(input_size=180,hidden_size=90,num_layers =10,dropout = 0.2).cuda()
        self.fc3 = nn.Linear(90,90).cuda()
        
    def forward(self, x):
        ret = self.relu(self.conv1(x))
        ret = self.maxpool(ret.transpose_(1,2))
        ret = self.relu(self.conv2(ret.transpose_(1,2)))
        ret = self.maxpool(ret.transpose_(1,2))
        ret = self.fc1(ret.reshape(360,8))
        ret = self.fc2(ret)
        ret, hn1 = self.gru1(ret.reshape(1,1,360))
        ret, hn2 = self.gru2(ret)
        ret, hn3 = self.gru3(ret)
        for _ in range(3):
            ret = self.fc3(ret)
        return ret

## DataLoader

In [76]:
print(len(under210), len(over210), len(over390))

27 34 37


In [181]:
def normalization(x):
    mx = x.max()
    mn = x.min()
    leng = mx-mn
    return np.array([(i-mn)/leng for i in x])

In [187]:
path = './dataset/'
tokens = os.listdir(path)
isFile = re.compile(r'.*_price.h5')
over450 = []
for token in tokens:
    if re.match(isFile, token):
        price = pd.read_hdf(path+token)
        price = price.dropna()
        if price.shape[0]>=450: 
            over450.append(token)
            
files = []
for file in over390:
    locals()[file[:-9]] = pd.read_hdf(path+file)
    files.append(file[:-9])
    
cols = ['HIGH', 'CLOSE', 'OPEN', 'LOW']
device = torch.device('cuda')
for file in files:
    locals()[file] = locals()[file].dropna()
    locals()[file] = locals()[file].reset_index()
    
    a = normalization(locals()[file]['amount'].values)
    b = normalization(locals()[file][cols].values)
    c = normalization(locals()[file]['count'].values)

    xa = np.array([[value for _ in range(4)] for value in a[90:450]])
    xc = np.array([[value for _ in range(4)] for value in c[90:450]])
    temp = []
    temp.append(b[90:450])
    temp.append(xa)
    temp.append(xc)
    x = np.array(temp)
    x = x.reshape(1,3,360,4)
    locals()[file+'_x'] = Variable(torch.tensor(x, dtype=torch.float32, device=device), requires_grad=True).cuda()
    
    locals()[file+'_y'] = Variable(torch.tensor(a[:90], dtype=torch.float32, device=device), requires_grad=True).cuda()

## Train

In [203]:
model = CNN_GRU()
adam = torch.optim.Adam(model.parameters(),lr=0.001)
mse = nn.MSELoss()

epoches = 5

In [204]:
%%time
for epoch in range(epoches):
    for file in files:
        x = locals()[files[0]+'_x']
        y = locals()[files[0]+'_y']
        adam.zero_grad()
        out = model(x)
        loss = mse(out, y)
        loss.backward(retain_graph=True)
        adam.step()
    print(loss)

tensor(0.0004, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(4.7103e-05, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(8.9956e-06, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(3.1794e-06, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(4.9699e-06, device='cuda:0', grad_fn=<MeanBackward1>)
Wall time: 6.78 s


In [206]:
out.sum()

tensor(4.5096, device='cuda:0', grad_fn=<SumBackward0>)

## Conv Test

In [152]:
device = torch.device('cuda')
x = ae[cols][90:450].values
x = x.reshape(360,1,6)
h = np.ones((10,1,1))
h2 = np.ones((10,1,90))
x = Variable(torch.tensor(x, dtype=torch.float32, device=device), requires_grad=True)
h = Variable(torch.tensor(h, dtype=torch.float32, device=device), requires_grad=True)
h2 = Variable(torch.tensor(h2, dtype=torch.float32, device=device), requires_grad=True)
gru = nn.GRU(input_size=6,hidden_size=1,num_layers =10,dropout = 0.5).cuda()
out, hn = gru(x, h)
gru = nn.GRU(input_size=360,hidden_size=90,num_layers =10,dropout = 0.5).cuda()
out = Variable(out.reshape(1,1,360), requires_grad=True)
out, hn = gru(out, h2)
fc = nn.Linear(90,90).cuda()
fc(out)

ValueError: cannot reshape array of size 1440 into shape (360,1,6)

In [153]:
ae = ae.dropna()
cols = ae.columns.tolist()[2:-1]
y = ae['amount'][:90].values
x = ae[cols][90:450].values
xa = ae['amount'][90:450].values
xc = ae['count'][90:450].values
xa = np.array([[value for _ in range(4)] for value in xa])
xc = np.array([[value for _ in range(4)] for value in xc])
temp = []
temp.append(x)
temp.append(xa)
temp.append(xc)
x = np.array(temp)
x = x.reshape(1,3,360,4)
device = torch.device('cuda')
x = Variable(torch.tensor(x, dtype=torch.float32, device=device), requires_grad=True).cuda()

In [156]:
conv1 = nn.Conv2d(3, 64, 3, stride=1, padding=1).cuda()
conv2 = nn.Conv2d(32, 16, 3, stride=1, padding=1).cuda()
relu = nn.ReLU().cuda()
maxpool = nn.MaxPool2d((2,2)).cuda()
fc1 = nn.Linear(8,4).cuda()
fc2 = nn.Linear(4,1).cuda()
gru = nn.GRU(input_size=360,hidden_size=90,num_layers =10,dropout = 0.5).cuda()
fc3 = nn.Linear(90,90).cuda()

In [157]:
ret= relu(conv1(x))
ret = ret.transpose_(1,2)
ret = maxpool(ret)
ret = ret.transpose_(1,2)
ret = relu(conv2(ret))
ret = ret.transpose_(1,2)
ret = maxpool(ret)
ret = ret.reshape(360,8)
ret = fc1(ret)
ret = fc2(ret)
ret = ret.reshape(1,1,360)
ret, hn = gru(ret)
ret = fc3(ret)

In [161]:
y = Variable(torch.tensor(y, dtype=torch.float32, device=device), requires_grad=True).cuda()
adam.zero_grad()
loss = mse(ret, y)
loss.backward(retain_graph=True)
adam.step()